In [7]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from tensorboardX import SummaryWriter
from tqdm import tqdm
import gc
from copy import deepcopy
from gym_swimmer import SwimmerEnv

import torch
import numpy as np
from torch import nn
import math
from models import *   
from train_barrier_swimmer import *
from core import generate_default_model_name

import scipy
from random import shuffle
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from collections import defaultdict

# train

In [8]:
from gym_swimmer import SwimmerEnv
from stable_baselines3 import PPO
from tqdm import tqdm

Env = SwimmerEnv
env = SwimmerEnv()
model = PPO.load("swimmer/best_model.zip")

bnn = DMLP(state_dim=Env.state_dim, action_dim=Env.action_dim, mode='straight')
bnn.to(device)
bnn.train()

name_dict = generate_default_model_name(Env)
# bnn.load_state_dict(torch.load(name_dict['b'].replace('.pt', '_1model.pt'), map_location=device))

boptimizer = torch.optim.Adam(bnn.parameters(), lr=1e-4, weight_decay=1e-8)
bscheduler = torch.optim.lr_scheduler.ExponentialLR(boptimizer, gamma=0.996)  

writer = SummaryWriter()
writer.step = 0
max_episode_length = 256
BATCH = 64
N_ITER = 1
N_EPOCH = 12000
N_TRAJ = 100

# threshold=1e-2
running_unsafe_rate = 0
best_unsafe_rate = float('inf')
unsafe_rates = [1.]*90

BMODEL_PATH = name_dict['db'].replace('dbgnn', 'dbnn')
open('1model_'+Env.__name__+'.txt', 'w+').close()
bbuf_gather = GatherReplayBuffer(BATCH)
cbuf = GlobalReplayBuffer()
fbuf = GlobalReplayBuffer()

pbar = tqdm(range(N_EPOCH))
for epoch_i in pbar:

    explore_eps = min(max(1-epoch_i/4000., 0.1), 0.5)
    nominal_eps = min(max(2-epoch_i/3000., 0.1), 1.0)
    bbuf = GlobalReplayBuffer()

    env.reset()
#     env.world.agents = env.world.sample_agents(env.num_agents, prob=0.0)
    total_trans=0; n_danger=0; threshold=1e-2

    while True:
        o = env._get_obs()
        a_best = model.predict(o)[0]
        noise = 0.01 * min(1., epoch_i/500.)
        # if noise != 1.:
        #     a_other = np.expand_dims(a_best, axis=0) + np.random.uniform(-noise, noise, size=(n_candidates-1, env.action_dim))
        #     a_other = a_other.clip(-1., 1.)
        # else:
        a_other = np.random.uniform(-1, 1, size=(n_candidates-1, env.action_dim))
        a_all = np.zeros((n_candidates, env.action_dim))
        a_all[0, :] = a_best
        a_all[1:, :] = a_other

        o = {'x': torch.FloatTensor(o), 'goal': torch.FloatTensor(env.goal)}
        a_refine, bvalue = iter_action(bnn, o, a_best.reshape(1, -1), max_iter=0, threshold=threshold)  #  min(epoch_i//100, 30)

        a, a_value = choose_action(a_refine, bvalue, threshold=threshold, explore_eps=explore_eps, nominal_eps=nominal_eps)

        next_o, rw, done, info = env.step(a)

        bbuf.store(**info,  bvalue=a_value, feasible=a_value>=threshold)
        prev_danger = info['prev_danger']
        next_danger = info['next_danger']
        if np.any(prev_danger) or np.any(next_danger):
            cbuf.store(**info, bvalue=1, feasible=a_value>=threshold)

        threshold = 1e-2
        total_trans += 1
        n_danger += np.array(next_danger).sum()

        if done or (total_trans >= max_episode_length):
            # if n_danger!=0:
            #     bbuf.relabel(-1e4 if epoch_i<100 else 5e-3)
            # else:
            #     fbuf.obs_buf.append(bbuf)
            bbuf_gather.append(bbuf)
            break

    unsafe_rate = n_danger / total_trans
    unsafe_rates.append(unsafe_rate)
    running_unsafe_rate = np.mean(unsafe_rates)
    if running_unsafe_rate < best_unsafe_rate:
        best_unsafe_rate = running_unsafe_rate
        torch.save(bnn.state_dict(), BMODEL_PATH)

    # if len(bbuf_gather.buffers) > N_TRAJ:
    #     bbuf_gather.buffers.pop(0)
    if len(unsafe_rates) > 100:
        unsafe_rates.pop(0)


    if epoch_i % 100 == 0:
        bnn.train()
        train_barrier(bnn, boptimizer, bbuf_gather, writer, pbar=pbar, n_iter=200)

    with open('1model_'+Env.__name__+'.txt', 'a+') as f:
        f.write(pbar.desc+'\t'+str(epoch_i)+'\t'+'unsafe rate: {0:.6f}'.format(unsafe_rate)+' running rate: {0:.6f}'.format(running_unsafe_rate)+', buf size {0:d} {1:d} {2:d}'.format(len(bbuf.obs_buf), len(cbuf.obs_buf), len(fbuf.obs_buf))+'\n')

    torch.save(bnn.state_dict(), BMODEL_PATH.replace('.pt', '_current.pt'))

bloss 0.002129, dloss 0.007798, closs 0.000000:   1%|          | 100/12000 [00:16<33:19,  5.95it/s]


KeyboardInterrupt: 

# train offline

In [ ]:
from gym_swimmer import SwimmerEnv
from stable_baselines3 import PPO
from tqdm import tqdm

Env = SwimmerEnv
env = SwimmerEnv()
model = PPO.load("swimmer/best_model.zip")

bnn = DMLP(state_dim=Env.state_dim, action_dim=Env.action_dim, mode='straight')
bnn.to(device)
bnn.train()

name_dict = generate_default_model_name(Env)
# bnn.load_state_dict(torch.load(name_dict['b'].replace('.pt', '_1model.pt'), map_location=device))

boptimizer = torch.optim.Adam(bnn.parameters(), lr=1e-4, weight_decay=1e-8)
bscheduler = torch.optim.lr_scheduler.ExponentialLR(boptimizer, gamma=0.996)  

writer = SummaryWriter()
writer.step = 0
max_episode_length = 256
BATCH = 64
N_ITER = 1
N_EPOCH = 12000
N_TRAJ = 1000
N_TRAJ_PER_EPOCH = 100
N_BUFFER = 100

threshold=1e-2
running_unsafe_rate = 0
best_unsafe_rate = float('inf')
unsafe_rates = [1.]*90
# nominal_eps = 1.0
explore_eps = 0.8

BMODEL_PATH = name_dict['db'].replace('dbgnn', 'dbnn')
open('1model_'+Env.__name__+'.txt', 'w+').close()
bbuf_gather = GatherReplayBuffer(BATCH)
cbuf = GlobalReplayBuffer()
fbuf = GlobalReplayBuffer()

pbar = tqdm(range(N_TRAJ))
for epoch_i in pbar:
    
    bbuf = GlobalReplayBuffer()

    env.reset()
#     env.world.agents = env.world.sample_agents(env.num_agents, prob=0.0)
    total_trans=0; n_danger=0; threshold=1e-2

    while True:
        o = env._get_obs()
        a_best = model.predict(o)[0]
        noise = 0.01 * min(1., epoch_i/500.)
        # if noise != 1.:
        #     a_other = np.expand_dims(a_best, axis=0) + np.random.uniform(-noise, noise, size=(n_candidates-1, env.action_dim))
        #     a_other = a_other.clip(-1., 1.)
        # else:
        a_other = np.random.uniform(-1, 1, size=(n_candidates-1, env.action_dim))
        a_all = np.zeros((n_candidates, env.action_dim))
        a_all[0, :] = a_best
        a_all[1:, :] = a_other

        o = {'x': torch.FloatTensor(o), 'goal': torch.FloatTensor(env.goal)}
        a_refine, bvalue = iter_action(bnn, o, a_all, max_iter=0, threshold=threshold)  #  min(epoch_i//100, 30)

        a, a_value = choose_action(a_refine, bvalue, threshold=threshold, explore_eps=0.05*(1+(epoch_i%20)), nominal_eps=0.01*(1+(epoch_i%100)))  # 

        next_o, rw, done, info = env.step(a)

        bbuf.store(**info,  bvalue=a_value, feasible=a_value>=threshold, collision=False)
        prev_danger = info['prev_danger']
        next_danger = info['next_danger']
        if next_danger:
            cbuf.store(**info, bvalue=1, feasible=a_value>=threshold, collision=False)

        threshold = 1e-2
        total_trans += 1
        n_danger += np.array(next_danger).sum()

        if done or (total_trans >= max_episode_length):
            if n_danger!=0:
                for o in bbuf.obs_buf:
                    o['prev_free'] = 0 * o['prev_free']
                    o['next_free'] = 0 * o['next_free']
                # TODO: not feasible if next state is not feasible
                # for o in bbuf.obs_buf:
            bbuf_gather.append(bbuf)
            # else:
            #     fbuf.obs_buf.append(bbuf)
            break

    unsafe_rate = n_danger / total_trans
    unsafe_rates.append(unsafe_rate)
    running_unsafe_rate = np.mean(unsafe_rates)
    if running_unsafe_rate < best_unsafe_rate:
        best_unsafe_rate = running_unsafe_rate
        torch.save(bnn.state_dict(), BMODEL_PATH)

    if len(bbuf_gather.buffers) > N_BUFFER:
        bbuf_gather.buffers.pop(0)
    if len(unsafe_rates) > N_TRAJ_PER_EPOCH:
        unsafe_rates.pop(0)

    if (epoch_i % N_TRAJ_PER_EPOCH) == (N_TRAJ_PER_EPOCH-1):
        bnn.train()
        train_barrier(bnn, boptimizer, bbuf_gather, writer, pbar=pbar, n_iter=200)
        torch.save(bnn.state_dict(), BMODEL_PATH.replace('.pt', '_current.pt'))

    with open('1model_'+Env.__name__+'.txt', 'a+') as f:
        f.write(pbar.desc+'\t'+str(epoch_i)+'\t'+'unsafe rate: {0:.6f}'.format(unsafe_rate)+' running rate: {0:.6f}'.format(running_unsafe_rate)+', buf size {0:d} {1:d} {2:d}'.format(len(bbuf.obs_buf), len(cbuf.obs_buf), len(fbuf.obs_buf))+'\n')

iter #118, bloss 0.000000, dloss 0.003855, closs 0.000000: 100%|█████████▉| 999/1000 [47:06<00:00,  2.93it/s]   

In [4]:
%debug

> /home/rainorangelemon/Documents/CAM/train_barrier_swimmer.py(178)collate_fn()
    176                      and label/length are scalars
    177             """
--> 178             o, next_o = data
    179             datas = []
    180             for data in [o, next_o]:



ipdb>  data


[({'x': tensor([ 0.0608, -0.0564, -0.0698, -0.0187, -0.0942, -0.0371,  0.0537, -0.0017,
         0.0610, -0.0459]), 'next_x': tensor([-0.8761, -0.5102, -1.6849,  1.7462,  0.9322, -0.9144, -0.0504, -1.0138,
        -0.0093,  2.9787]), 'action': tensor([0.9804, 0.6299]), 'goal': tensor([4., 0.]), 'prev_free': tensor(1.), 'prev_danger': tensor(0.), 'next_goal': tensor(0.), 'next_free': tensor(1.), 'next_danger': tensor(0.), 'bvalue': tensor(-0.0559), 'feasible': tensor(0.)}, {'x': tensor([-0.8761, -0.5102, -1.6849,  1.7462,  0.9322, -0.9144, -0.0504, -1.0138,
        -0.0093,  2.9787]), 'next_x': tensor([-0.1244, -0.2783, -0.1126, -1.0883,  1.7440, -0.2624,  1.6279,  2.6725,
        -4.4296, -0.0209]), 'action': tensor([-0.8883, -0.0444]), 'goal': tensor([4., 0.]), 'prev_free': tensor(1.), 'prev_danger': tensor(0.), 'next_goal': tensor(0.), 'next_free': tensor(1.), 'next_danger': tensor(0.), 'bvalue': tensor(-0.0683), 'feasible': tensor(0.)}), ({'x': tensor([-0.8761, -0.5102, -1.6849,  1.

ipdb>  len(data)


64


ipdb>  data[0][0]


{'x': tensor([ 0.0608, -0.0564, -0.0698, -0.0187, -0.0942, -0.0371,  0.0537, -0.0017,
         0.0610, -0.0459]), 'next_x': tensor([-0.8761, -0.5102, -1.6849,  1.7462,  0.9322, -0.9144, -0.0504, -1.0138,
        -0.0093,  2.9787]), 'action': tensor([0.9804, 0.6299]), 'goal': tensor([4., 0.]), 'prev_free': tensor(1.), 'prev_danger': tensor(0.), 'next_goal': tensor(0.), 'next_free': tensor(1.), 'next_danger': tensor(0.), 'bvalue': tensor(-0.0559), 'feasible': tensor(0.)}


ipdb>  len(data[0])


2


ipdb>  exit()


In [702]:
n_d = 0
for d in cbuf.obs_buf:
    if not d['prev_danger']:
        n_d += 1
        print('--------------------------------')
        print(bnn(x=d['x'].to(device), action=d['action'].to(device)))
        for _ in range(10):
            a = torch.FloatTensor(2).uniform_(-1, 1)
            print(bnn(x=d['x'].to(device), action=a.to(device)))
print(n_d)

--------------------------------
tensor(-0.1596, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(0.0054, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(0.0502, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(0.0490, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(-0.1516, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(-0.1665, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(0.0500, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(-0.0523, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(0.0514, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(-0.1572, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(0.0486, device='cuda:0', grad_fn=<SqueezeBackward1>)
--------------------------------
tensor(-0.0143, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(0.0372, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(0.0068, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(0.0515, device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor(0.0312, device='cuda:

KeyboardInterrupt: 

In [451]:
bnn(x=cbuf.obs_buf[0]['x'].to(device), action=cbuf.obs_buf[0]['action'].to(device))

tensor(-0.1215, device='cuda:0', grad_fn=<SqueezeBackward1>)

In [415]:
cbuf.obs_buf[0]['action'].to(device)

tensor([-0.8627, -1.0000], device='cuda:0')

In [434]:
torch.rand(2).uniform_(-1, 1)

tensor([-0.5567,  0.0523])

## check on existing data

In [547]:
total_buffer = []
total_buffer_a = []
total_buffer_d = []
total_buffer.extend([o['x'].data.cpu().numpy() for b in bbuf_gather.buffers for o in b.obs_buf])
total_buffer_a.extend([o['action'].data.cpu().numpy() for b in bbuf_gather.buffers for o in b.obs_buf])
total_buffer_d.extend([o['next_danger'].data.cpu().numpy() for b in bbuf_gather.buffers for o in b.obs_buf])

In [453]:
len(total_buffer)

32830

In [454]:
len(cbuf.obs_buf)

103

In [392]:
total_buffer = np.array(total_buffer)
total_buffer_a = np.array(total_buffer_a)

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(total_buffer)
dists, idxs = nbrs.kneighbors(total_buffer, 10, return_distance=True)
best_idx = dists.mean(axis=-1).argmin()
random_data = total_buffer[best_idx]
dists, idxs = nbrs.kneighbors([random_data], 10, return_distance=True)
total_buffer_a[idxs], dists

# inference

In [613]:
from gym_swimmer import SwimmerEnv
from stable_baselines3 import PPO
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Env = SwimmerEnv
env = SwimmerEnv()
model = PPO.load("swimmer/best_model.zip")
name_dict = generate_default_model_name(Env)

bnn = DMLP(state_dim=Env.state_dim, action_dim=Env.action_dim, mode='straight')
BMODEL_PATH = name_dict['db'].replace('dbgnn', 'dbnn').replace('.pt', '_current.pt')
bnn.load_state_dict(torch.load(BMODEL_PATH, map_location=device))
bnn.to(device)
bnn.eval()

writer = SummaryWriter()
writer.step = 0
max_episode_length = 256
BATCH = 64
N_ITER = 1
N_EPOCH = 12000

In [671]:
n_collide = 0
n_reach = 0
threshold = 1000
N_TEST = 1000
for _ in range(N_TEST):
    print('------------------------'+str(_)+'------------')
    env.reset()
    #     env.world.agents = env.world.sample_agents(env.num_agents, prob=0.0)
    total_trans=0; n_danger=0; threshold=1000

    while True:
        o = env._get_obs()
        a_best = model.predict(o)[0]
        a_other = np.random.uniform(-1., 1., size=(n_candidates-1, env.action_dim))
        a_all = np.zeros((n_candidates, env.action_dim))
        a_all[0, :] = a_best
        a_all[1:, :] = a_other

        o = {'x': torch.FloatTensor(o), 'goal': torch.FloatTensor(env.goal)}
        a_refine, bvalue = iter_action(bnn, o, a_other, max_iter=0, threshold=threshold)

        a, a_value = choose_action(a_refine, bvalue, threshold=threshold, explore_eps=0., nominal_eps=0.)
        # if bvalue[0] > 1e-2:
        #     a = a_refine[0]
        # elif bvalue.max() > 1e-2:
        #     a = a_refine[bvalue.argmax()]
        # else:
        #     a = a_refine[0]
        
        threshold = max(1e-2, 1e-2+0.9*a_value)

        next_o, rw, done, info = env.step(a_all[np.random.randint(len(a_all))])
        if info['next_danger']:
            print(bvalue[0], bvalue.min(), bvalue.max(), threshold, a_value, 'TRUE' if info['next_danger'] else '')
        n_collide += info['next_danger']

        threshold = 1e-2
        total_trans += 1
        n_danger += np.array(next_danger).sum()

        if done or (total_trans >= max_episode_length):
            n_reach += info['next_goal']
            break

print(n_collide, n_reach, info['next_goal'])

------------------------0------------
------------------------1------------
------------------------2------------
------------------------3------------
------------------------4------------
------------------------5------------
------------------------6------------
------------------------7------------
------------------------8------------
0.05749713 0.056948084 0.05837439 0.06174741595983506 0.05749713 TRUE
------------------------9------------
0.055106763 -0.10493525 0.074837424 0.05959608651697636 0.055106763 TRUE
------------------------10------------
------------------------11------------
0.054704163 0.054039586 0.055297107 0.05923374690115452 0.054704163 TRUE
------------------------12------------
------------------------13------------
0.051981583 -0.10237061 0.0705899 0.05678342491388321 0.051981583 TRUE
------------------------14------------
------------------------15------------
0.014792301 -0.23730448 0.055694237 0.0233130706846714 0.014792301 TRUE
------------------------16-

In [672]:
n_collide

424

In [701]:
n_collide = 0
n_reach = 0
threshold = 1000
N_TEST = 1000
imgs = [env.sim.render(600, 300)]
env.reset()
#     env.world.agents = env.world.sample_agents(env.num_agents, prob=0.0)
total_trans=0; n_danger=0; threshold=1e-2

while True:
    o = env._get_obs()
    a_best = model.predict(o)[0]
    a_other = np.random.uniform(-1., 1., size=(n_candidates-1, env.action_dim))
    a_all = np.zeros((n_candidates, env.action_dim))
    a_all[0, :] = a_best
    a_all[1:, :] = a_other

    o = {'x': torch.FloatTensor(o), 'goal': torch.FloatTensor(env.goal)}
    a_refine, bvalue = iter_action(bnn, o, a_other, max_iter=0, threshold=threshold)

    a, a_value = choose_action(a_refine, bvalue, threshold=threshold, explore_eps=0., nominal_eps=0.)
    # if bvalue[0] > 1e-2:
    #     a = a_refine[0]
    # elif bvalue.max() > 1e-2:
    #     a = a_refine[bvalue.argmax()]
    # else:
    #     a = a_refine[0]

    # threshold = max(1e-2, 1e-2+0.9*a_value)

    next_o, rw, done, info = env.step(a)
    imgs.append(env.sim.render(600, 300))
    # if info['next_danger']:
    print(bvalue[0], bvalue.min(), bvalue.max(), threshold, a_value, 'TRUE' if info['next_danger'] else '')
    n_collide += info['next_danger']

    threshold = 1e-2
    total_trans += 1
    n_danger += np.array(next_danger).sum()

    if done or (total_trans >= max_episode_length):
        n_reach += info['next_goal']
        break
        
        
from PIL import Image
ims = [Image.fromarray(np.flip(a_frame, axis=0)) for a_frame in imgs]
ims[0].save("swimmer_cam.gif", save_all=True, append_images=ims[1:], duration=20)

0.020860227 0.007124327 0.028603137 0.01 0.020860227 
0.05027492 0.048380457 0.060478203 0.01 0.05027492 
0.0507154 -0.0024082512 0.0993887 0.01 0.0507154 
0.051314097 0.0500434 0.052226078 0.01 0.051314097 
0.050846525 0.050261643 0.05198429 0.01 0.050846525 
0.05104264 0.050819244 0.051897638 0.01 0.05104264 
0.050763432 0.0504603 0.056792505 0.01 0.050763432 
0.0511712 0.05038765 0.06666562 0.01 0.0511712 
0.052341707 0.049571417 0.053833824 0.01 0.052341707 
0.053486492 0.0064754114 0.07849544 0.01 0.053486492 
0.050061155 0.047322556 0.05420212 0.01 0.050061155 
0.051518872 0.049978394 0.053684082 0.01 0.051518872 
0.050684728 0.04981617 0.053300563 0.01 0.050684728 
0.051234998 0.05064484 0.059528474 0.01 0.051234998 
0.050862234 0.008137327 0.054674685 0.01 0.050862234 
0.0510889 0.050590616 0.05216887 0.01 0.0510889 
0.052022554 0.051325828 0.053494427 0.01 0.052022554 
0.053260945 0.051509473 0.0538807 0.01 0.053260945 
0.050318222 0.050173312 0.05848288 0.01 0.050318222 
0.05